In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
from sentence_transformers import SentenceTransformer

# Load dataset
df = pd.read_csv("/mnt/data/Dataset_Python_Question_Answer.csv")
print("Dataset Loaded. Shape:", df.shape)
print(df.head())

# Load trained model
trained_model = tf.keras.models.load_model("/mnt/data/siamese_lstm_model.h5")
print("Trained Model Loaded.")

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Load SBERT model
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to get predictions
def get_predictions(model, tokenizer, texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=1).numpy()
    return predictions

# Get predictions for each model
sample_texts = df['Question'].astype(str).tolist()[:100]  # Taking a sample for evaluation
bert_preds = get_predictions(bert_model, bert_tokenizer, sample_texts)
sbert_preds = np.random.randint(0, 2, size=len(sample_texts))  # Placeholder for SBERT
trained_model_preds = np.random.randint(0, 2, size=len(sample_texts))  # Placeholder for trained model

y_true = np.random.randint(0, 2, size=len(sample_texts))  # Placeholder true labels

# Generate classification reports
bert_report = classification_report(y_true, bert_preds, output_dict=True)
sbert_report = classification_report(y_true, sbert_preds, output_dict=True)
trained_model_report = classification_report(y_true, trained_model_preds, output_dict=True)

# Plot classification metrics
def plot_metrics(report, title):
    df_report = pd.DataFrame(report).transpose()
    df_report.drop(["accuracy"], inplace=True, errors='ignore')
    df_report.plot(kind='bar', figsize=(10, 6))
    plt.title(title)
    plt.xlabel("Metrics")
    plt.ylabel("Score")
    plt.ylim(0, 1)
    plt.show()

plot_metrics(bert_report, "BERT Classification Report")
plot_metrics(sbert_report, "SBERT Classification Report")
plot_metrics(trained_model_report, "Trained Model Classification Report")
